In [1]:
try:
    import dataretrieval
except ImportError:
    print("dataretrieval not found. Installing...")
    !pip install dataretrieval
    import dataretrieval # Import again after installation

from dataretrieval import nwis # hydrological time-series data from USGS

# Use scikit-learn to grid search the number of neurons
!pip install scikit-learn==1.5.2 # TO AVOID AttributeError: 'super' object has no attribute '__sklearn_tags__'
import sklearn
print(sklearn.__version__)

from sklearn.model_selection import GridSearchCV  #  hyperparameter tuning
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.models import Sequential

#!pip install scikeras
#from scikeras.wrappers import KerasRegressor

try:
    from scikeras.wrappers import KerasRegressor
except ImportError:
    print("scikeras not found. Installing...")
    !pip install scikeras
    from scikeras.wrappers import KerasRegressor

from tensorflow.keras.constraints import MaxNorm
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # Imports several common regression evaluation metrics from scikit-learn:
from datetime import datetime
# Get the current time
current_time = datetime.now()
# Print the current time
print("Current time:", current_time)

1.5.2
Current time: 2025-06-26 01:28:38.539106


In [2]:
startDate = "1995-10-01"
endDate = "2024-09-30"
model_site = [
  "13190500"
]

#units_day = 365
#missing_data_threshold = 0.90 *365

summer_months = [6,7,8]
#summer_missing_data_threshold = 0.90 * (30+31+31)

In [3]:
site_info = nwis.get_info(sites= model_site )
site_info_df = pd.DataFrame( site_info[0] )
print("site_info_df:",site_info_df)
print("station_nm:",site_info_df['station_nm'][0])
site_info_df["site_no"]=site_info_df["site_no"].astype(str)
columns_to_save = ['site_no', 'station_nm','dec_lat_va', 'dec_long_va', 'drain_area_va', 'contrib_drain_area_va' ,   'alt_va']
filtered_df = site_info_df[columns_to_save]

parameterCode = ["00010","00060"  ]
dailyStreamtemp = nwis.get_dv(sites= model_site
                              , parameterCd=parameterCode, start=startDate, end=endDate)
dailyStreamtemp_df = pd.DataFrame( dailyStreamtemp[0] )

dailyStreamtemp_df = dailyStreamtemp_df.reset_index()
# Extracts Date, Month, and Year components from the datetime column.
dailyStreamtemp_df['Date'] = dailyStreamtemp_df['datetime'].dt.date
dailyStreamtemp_df['Month'] = dailyStreamtemp_df['datetime'].dt.month
dailyStreamtemp_df['Year'] = dailyStreamtemp_df['datetime'].dt.year

dailyStreamtemp_df = dailyStreamtemp_df[ ~( (dailyStreamtemp_df["site_no"]=="13310800") & (dailyStreamtemp_df["Year"] == (2015) ) ) ]

shared_url = "https://drive.google.com/file/d/1z9w4rk21CCbbuOjwsru1pgkiOsYgVX6h/view?usp=share_link"
file_id = shared_url.split('/d/')[1].split('/')[0]
download_url = f"https://drive.google.com/uc?id={file_id}"
prism_df = pd.read_csv(download_url)
prism_df['date'] = pd.to_datetime( prism_df['date'])

# https://www.tensorflow.org/tutorials/structured_data/time_series
# https://www.scirp.org/journal/paperinformation?paperid=142149
# https://etasr.com/index.php/ETASR/article/view/10595

def days_since_october_first(date):
    october_first = pd.Timestamp(year=date.year, month=10, day=1)
    days_difference = (date - october_first).days
    return days_difference if days_difference >= 0 else days_difference + 365

prism_df['Days_Since_October_1st'] =  prism_df['date'].apply(days_since_october_first)

prism_df["site_no"] =prism_df["site_no"].astype(str)
# Add a leading 0 to strings with exactly 7 characters
prism_df['site_no'] = prism_df['site_no'].apply(lambda x: '0' + x if len(x) == 7 else x)
prism_df["site_no"] =prism_df["site_no"].astype(str)

prism_df['Date'] = prism_df['date'].dt.date

prism_df = prism_df[['tmean' , "site_no", 'Date','Days_Since_October_1st'
 ]]

dailyStreamtemp_df = dailyStreamtemp_df[[    'Date',       'site_no',  '00010_Mean', '00060_Mean'
   ]]

result = pd.merge(prism_df, dailyStreamtemp_df, on=['Date','site_no'], how='inner')

# Count the number of NaN values in each column
na_count = result.isna().sum()

# Drop rows with any NaN values
df_cleaned = result.dropna()

df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])

train_month =  list(range( 3 , 8+1 ))

print("train_month:", train_month)

df_cleaned= df_cleaned[ df_cleaned['Date'].dt.month.isin( train_month )]

print("df_cleaned:",df_cleaned)

print(" df_cleaned.describe():",df_cleaned.describe())

# Count the number of NaN values in each column
na_count = df_cleaned.isna().sum()

print(na_count)

# gap

# Calculate time difference between consecutive dates
df_cleaned['Gap'] = df_cleaned['Date'].diff()

# Filter rows where the gap is more than expected (e.g., > 1 day)
expected_freq = pd.Timedelta(days=1)
gaps = df_cleaned[df_cleaned['Gap'] > expected_freq]

print("gaps",gaps)

site_info_df:   agency_cd   site_no                               station_nm site_tp_cd  \
0      USGS  13190500  SF BOISE RIVER AT ANDERSON RANCH DAM ID         ST   

   lat_va  long_va  dec_lat_va  dec_long_va coord_meth_cd coord_acy_cd  ...  \
0  432037  1152839   43.343611    -115.4775             G            S  ...   

  reliability_cd gw_file_cd  nat_aqfr_cd  aqfr_cd  aqfr_type_cd well_depth_va  \
0            NaN   NNNNNNNN          NaN      NaN           NaN           NaN   

  hole_depth_va depth_src_cd  project_no                    geometry  
0           NaN          NaN         NaN  POINT (-115.4775 43.34361)  

[1 rows x 43 columns]
station_nm: SF BOISE RIVER AT ANDERSON RANCH DAM ID
train_month: [3, 4, 5, 6, 7, 8]
df_cleaned:            tmean   site_no       Date  Days_Since_October_1st  00010_Mean  \
8918   -1.199000  13190500 2020-03-01                     151         4.0   
8919   -1.367000  13190500 2020-03-02                     152         4.1   
8920    3.940400 

/tmp/ipython-input-3-3487914925.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])


In [4]:
# --- 2. Preprocessing ---
# Select features and target
features = ['tmean',  '00060_Mean' ,"Days_Since_October_1st"]
target = '00010_Mean'
# Normalize the data
# It's crucial to normalize all input features and the target variable for LSTMs.
# We'll use a separate scaler for the target 'y' to easily inverse transform predictions.
#  the minimum of feature is made equal to zero and the maximum of feature equal to one.
scaler_features = MinMaxScaler(feature_range=(0, 1)) # other scalers like StandardScaler (Z-score normalization) might be more appropriate
scaler_target = MinMaxScaler(feature_range=(0, 1))

print(df_cleaned.shape)
df = df_cleaned.copy()
print(df.shape)
# Fit and transform features
df[features] = scaler_features.fit_transform(df[features])

# Fit and transform target
# Reshape for scaler (expects 2D array)
df[target] = scaler_target.fit_transform(df[target].values.reshape(-1, 1))
print("\nDataFrame after Normalization Head:")
print(df.tail())

(920, 7)
(920, 7)

DataFrame after Normalization Head:
          tmean   site_no       Date  Days_Since_October_1st  00010_Mean  \
10558  0.722937  13190500 2024-08-27                0.978142    0.744186   
10559  0.607814  13190500 2024-08-28                0.983607    0.720930   
10560  0.641404  13190500 2024-08-29                0.989071    0.709302   
10561  0.737673  13190500 2024-08-30                0.994536    0.732558   
10562  0.842329  13190500 2024-08-31                1.000000    0.732558   

       00060_Mean    Gap  
10558    0.048342 1 days  
10559    0.042626 1 days  
10560    0.042789 1 days  
10561    0.042300 1 days  
10562    0.042300 1 days  


In [5]:
# --- 3. Create Sequences for LSTM ---
# LSTMs require data in a 3D format: (samples, timesteps, features)
# 'timesteps' (also known as look_back) determines how many previous time steps
# the LSTM will consider to predict the next value.

look_back =  90 # You can adjust this based on your data's seasonality/dependencies

def create_sequences(data, look_back, features_cols, target_col):
    X, Y = [], []
    # Ensure data is a numpy array for efficient indexing
    data_np = data[features_cols + [target_col]].values # include target for sequence alignment
    print("data_np:",data_np)
    # Features are the first 'num_features' columns, target is the last column.
    num_features = len(features_cols)

    for i in range(len(data_np) - look_back):
        # X: look_back historical features (x1, x2)
        X.append(data_np[i:(i + look_back +1), :num_features]) # (i + look_back +1) will include the feature of the day of the target
        # Y: the 'y' value at the current time step (i + look_back)
        Y.append(data_np[i + look_back, num_features]) # Target is the last column
    return np.array(X), np.array(Y)

X, Y = create_sequences(df, look_back, features, target)

print("X", X) # did I miss today's X?

# Print first 4 elements
print("First 4 Y elements:", Y[:4])

# Print last 4 elements
print("Last 4 Y elements:", Y[-4:])

print(f"\nShape of X (samples, timesteps, features): {X.shape}")
print(f"Shape of Y (samples, target_value): {Y.shape}")

data_np: [[0.19632916 0.00620611 0.         0.06976744]
 [0.19166214 0.0052262  0.00546448 0.08139535]
 [0.33910138 0.00571615 0.01092896 0.11627907]
 ...
 [0.64140402 0.04278948 0.98907104 0.70930233]
 [0.73767264 0.04229953 0.99453552 0.73255814]
 [0.84232937 0.04229953 1.         0.73255814]]
X [[[0.19632916 0.00620611 0.        ]
  [0.19166214 0.0052262  0.00546448]
  [0.33910138 0.00571615 0.01092896]
  ...
  [0.73914915 0.14421035 0.48087432]
  [0.85649158 0.14584354 0.4863388 ]
  [0.82773935 0.14421035 0.49180328]]

 [[0.19166214 0.0052262  0.00546448]
  [0.33910138 0.00571615 0.01092896]
  [0.30857123 0.00555283 0.01639344]
  ...
  [0.85649158 0.14584354 0.4863388 ]
  [0.82773935 0.14421035 0.49180328]
  [0.63928026 0.14421035 0.49726776]]

 [[0.33910138 0.00571615 0.01092896]
  [0.30857123 0.00555283 0.01639344]
  [0.33729846 0.00457292 0.02185792]
  ...
  [0.82773935 0.14421035 0.49180328]
  [0.63928026 0.14421035 0.49726776]
  [0.66326826 0.14584354 0.50273224]]

 ...

 [[0.

In [6]:
# --- 4. Train-Test Split (Time Series Split) ---
# It's crucial to split time series data chronologically to avoid data leakage.
train_size = int(len(X) * 0.8) # e.g., 80% for training, 20% for testing
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(Y)]

print(f"Train samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f" Y Train samples: {len(Y_train)}")
print(f"Y Test samples: {len(Y_test)}")

Train samples: 664
Test samples: 166
 Y Train samples: 664
Y Test samples: 166


In [7]:
learning_rate = 0.001

def create_model( h1,   d1 , h2, d2,learning_rate ):
    # create model
    model = Sequential()
    model.add(LSTM(units= h1, return_sequences=True , input_shape=(look_back, len(features))))
    model.add(Dropout(  d1 ))  # Dropout to prevent overfitting
    model.add(LSTM(units=h2, return_sequences=False))  # Last LSTM layer
    model.add(Dropout(d2))
    model.add(Dense(units=1))
    optimizer = Adam(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# fix random seed for reproducibility
seed = 42
tf.random.set_seed(seed)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) #Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
max_epochs =34

# create model
model = KerasRegressor(model=create_model
                       , epochs=  max_epochs
                       , callbacks=[early_stop]
                      , validation_split=0.2
                       #, batch_size=10
                       , verbose=0)
print(model.get_params().keys())
print("model:",model)
# define the grid search parameters
h1 = [     2,4
      #,8,16,32,64,128,256
           ]
h2 = [     2,4
#      ,8,16,32,64,128,256
           ]
d1 = [0.0,0.05
      #, 0.1,0.2
      ]
d2 = [ 0.0,0.05
      #, 0.1,0.2
       ]
learning_rate=[
    #0.0001,
    0.001,  0.01]
batch_size = [
    #4,8,
              16,32
               ]

param_grid = dict( model__h1= h1 ,model__d1= d1, model__h2=h2 , model__d2=d2 , model__learning_rate= learning_rate, batch_size=batch_size )
print(param_grid)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1
                    #, cv=3
                    )
print(grid)
grid_result = grid.fit(X_train, Y_train)
print("grid_result:",grid_result)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs'])
model: KerasRegressor(
	model=<function create_model at 0x7cbfabb55120>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=0
	callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7cbfab2872d0>]
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=34
)
{'model__h1': [2, 4], 'model__d1': [0.0, 0.05], 'model__h2': [2, 4], 'model__d2': [0.0, 0.05], 'model__learning_rate': [0.001, 0.01], 'batch_size': [16, 32]}
GridSearchCV(estimator=KerasRegressor(callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7cbfab2872d0>], epochs=34, model=<function create_model at 0x7cbfabb55120>, validation_split=0.2, verbose=0),
             n_jobs=-1,
  

In [ ]:
# --- 8. Make Predictions ---
# Make predictions on the test set
best_params= grid_result.best_params_

print("best_params:",best_params)

#print("best_params:",best_params.keys() )

#print("best_params:",best_params['model__d1'] )

# Enable eager execution
#tf.config.experimental_run_eagerly(True)

best_model = create_model(  best_params['model__h1'], best_params['model__d1'], best_params['model__h2'], best_params['model__d2'] , best_params['model__learning_rate']   )

print(best_model.summary())

#early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True

print(early_stop)

#best_model = create_model(**{k: best_params[k] for k in ['model__h1', 'model__d1', 'model__h2', 'model__d2' ]})
best_model.fit(X_train, Y_train, epochs=100 # best_params['epochs']
               , callbacks=[early_stop]
               , batch_size=best_params['batch_size']
               )

Y_pred_scaled = best_model.predict(X_test)
print(f"\nShape of Y_pred_scaled: {Y_pred_scaled.shape}")
print(f"Shape of Y_test: {Y_test.shape}")
print(f"Shape of X_test: {X_test.shape}")

# Inverse transform the predictions and actual values to the original scale
Y_pred = scaler_target.inverse_transform(Y_pred_scaled)
Y_test_original = scaler_target.inverse_transform(Y_test.reshape(-1, 1))

r2 = r2_score( Y_test_original ,  Y_pred )

# Calculate RMSE
all_rmse = np.sqrt(mean_squared_error(Y_test_original, Y_pred))
print(f"Test RMSE: {all_rmse:.4f}")

bias =np.mean( Y_pred- Y_test_original)
print(f"Test bias: { bias:.4f}")

# KGE calculation using hydroeval
# Note: hydroeval's kge function returns KGE, r, alpha, beta by default
# kge_value, r_comp, alpha_comp, beta_comp = he.kge( Y_pred , Y_test_original)
#print( kge_value)

std_ratio = np.std( Y_pred ) / np.std( Y_test_original ) # standard deviation ratio
correlation = np.corrcoef( Y_pred.flatten() ,  Y_test_original.flatten() )[0, 1]
print("correlation", correlation)

print(np.std(Y_pred))
print(np.std(Y_test_original))

In [ ]:
print("df_cleaned.tail:",df_cleaned.tail)
print((len(Y)-Y_test.shape[0]  ))
print(len(Y) )
dt_ys = df_cleaned.iloc[  ( df_cleaned.shape[0]-Y_test.shape[0]  )  : df_cleaned.shape[0] ]#['Date']

dt_ys['Y_pred'] = Y_pred
dt_ys['Y_test_original']= Y_test_original

dt_ys[['Date','Y_pred', 	'Y_test_original']]

dt_ys['Date'] = pd.to_datetime( dt_ys['Date'])

print( "dt_ys.tail()", dt_ys.tail() )

print( dt_ys['Date'].dt.month.describe())

summer_df = dt_ys[ dt_ys['Date'].dt.month.isin( summer_months)]

print("summer_df.tail()",summer_df.tail())

summer_df.to_csv( '/content/drive/My Drive/LSTM_DOY/LSTM_predictions_DOY_grid_search/'+( model_site[0] + "summer_df.csv") , index=False )

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(summer_df['Y_test_original'], summer_df['Y_pred']))
print(f"Test RMSE: {rmse:.4f}")

In [ ]:
from numpy import array
!pip install permetrics
from permetrics.regression import RegressionMetric

## For 1-D array
y_true = array([3, -0.5, 2, 7])
y_pred = array([2.5, 0.0, 2, 8])

evaluator = RegressionMetric(y_true, y_pred)
print(evaluator.nash_sutcliffe_efficiency())

## For > 1-D array
y_true = array([[0.5, 1], [-1, 1], [7, -6]])
y_pred = array([[0, 2], [-1, 2], [8, -5]])

evaluator = RegressionMetric(y_true, y_pred)
print(evaluator.NSE(multi_output="raw_values"))